In [ ]:
import sys
sys.path.insert(1, 'utils')
from utils import *
from data_manipulation import *
pd.options.display.max_colwidth = 100

## Categorías elegidas para el estudio
* MLA1002 (televisores)
* MLA1055 (celulares y smartphones)
* MLA398582 (Heladeras)
* MLA1644 (Aires Acondicionados)
* MLA438566 (Consolas)
* MLA1652 (Notebooks)

#### Las categorías elegidas representan, aquellas que según mi criterio son muy populares por los usuarios de MeLi

#### Disclaimer: las observaciones y conclusiones fueron realizadas con los datos extraídos de la api en el momento que se realizó el notebook, podría haber pequeñas diferencias, dado que la api trae resultados por relevancia. Aún así no creo que sean notables los cambios

### Levanto los datos guardados _(si no existe el archivo hay que correr el WDE notebook)_

In [ ]:
results = pd.read_csv("data/training_data.csv")

# __Análisis__

#### __¿Qué hay disponible?__

In [ ]:
results.info()

#### Se ve que hay muchas variables númericas, así también cómo variables catégoricas, también parece que hay features que se pueden armar como por ejemplo cantidad de fotos, cantidad de cuotas o la marca

## Preguntas

#### Se proponen las siguientes preguntas, a modo de incentivo para encarar la exploración de publicaciones con descuentos, en algunos casos las preguntas no se responderán para todas las categorías. 

* ¿Cuántos productos hay con descuento para cada categoría?
* ¿Cómo se distribuyen los precios para cada categoría?
* ¿Cuál es el promedio de descuento en cada categoría?
* ¿Cómo es y cuanto influye en las ventas el porcentaje de descuento?
* ¿Cuales son las marcas con mayores descuentos por categoría?¶
* ¿Existen marcas o modelos preferidos por los usuarios?
* ¿Cuánto gastan los usuarios? y cómo lo hacen en función de las cuotas?
* ¿Cuáles son las categorías favoritas para comprar con descuento?
* ¿Cómo son las ventas según el stock inicial?
* ¿Cómo impacta en las ventas con descuento la cantidad de atributos que publican los vendedores y las variaciones que ofrecen?
* ¿Cómo son las ventas por la fotos con las que se publican?
* ¿Cómo es la correlación de variables para cada categoría?



In [ ]:
results.category.value_counts()

#### Sin utilizar el access token, se pudo acceder a aproximadamente 1000 resultados por cada categoría.

### Distingo mis publicaciones con descuento para el estudio

In [ ]:
results_disc = results[~results.original_price.isna()]
tvs = results_disc[results_disc.category == 'televisores']
celulares = results_disc[results_disc.category == 'celulares']
notebooks = results_disc[results_disc.category == 'notebooks']
heladeras = results_disc[results_disc.category == 'heladeras']
aires = results_disc[results_disc.category == 'aires']
consolas = results_disc[results_disc.category == 'consolas']
grouped_total = results.groupby('category')
grouped_results = results_disc.groupby('category')

# __Preguntas__

### __¿Cuántos productos hay con descuento para cada categoría?__

In [ ]:
results_disc.category.value_counts()

##### Las notebooks son los productos con mayor cantidad de descuentos, mientras que las consolas son las que menos descuento tienen

### __¿Cómo se distribuyen los precios para cada categoría?__

In [ ]:
#Separo en distintas categorias para facilitar la visualización
elec = results_disc[results_disc.category.isin(['televisores', 'notebooks'])]
elec2 = results_disc[results_disc.category.isin(['consolas', 'celulares'])]
elec3 = results_disc[results_disc.category.isin(['heladeras', 'aires'])]


In [ ]:
ax = sns.displot(elec, x="price", hue="category",  fill=True, bins=100, aspect=3)
ax.set(xlabel='Precio', ylabel='Cantidad de productos')
ax = sns.displot(elec2, x="price", hue="category", fill=True, bins=100, aspect=3)
ax.set(xlabel='Precio', ylabel='Cantidad de productos')
ax = sns.displot(elec3, x="price", hue="category", fill=True, bins=100, aspect=3)
ax.set(xlabel='Precio', ylabel='Cantidad de productos')
plt.show()

#### Se puede ver que hay mucha dispersión para una misma categoría, por ejemplo en el caso de los celulares o las notebooks, sin embargo esto puede deberse a dos cosas:
* Hay muchas gamas o modelos para una categoría, que generan este efecto tan disperso
* Hay muchos productos mal catalogados, que pueden estar afectando los resultados, por ejemplo carcasas de celulares, cargadores de notebooks o joysticks de consolas

##### Para eso puedo analizar que sucede con el desvío y los valores extremos

In [ ]:
grouped_results.price.describe()

#### Por ejemplo se puede ver que los celulares tienen una media de precio de aproximadamente ~21k pesos, pero el valor mínimo resulta ser de 3000 y el desvío es de casi 25k, lo que me da a pensar que puede ser que haya productos mal catalogados

In [ ]:
celulares[celulares.price < 9000].head().title

In [ ]:
consolas[consolas.price <= 4000].head().title

In [ ]:
notebooks[notebooks.price <= 40000].head().title

#### Analizando los resultados se puede ver que a pesar de que puede haber productos catalogados erróneamente, en realidad lo que sucede es que hay una gran espectro de modelos para una categoría, por lo cual no habrá que realizar una limpieza en los productos

### __¿Cuál es el promedio de descuento en cada categoría?__

In [ ]:
grouped_results.discount_per.describe()

#### El promedio de descuento es de aprox el 15% para múltiples categorías, donde las notebooks tienen mayor porcentaje de descuento. Analizando los cuartiles, los descuentos se reparten de manera similar para todas las categorías.

### __¿Cómo es y cuanto influye en las ventas el porcentaje de descuento?__

#### Televisores

In [ ]:
tvs_fig = sns.jointplot(data=tvs, x="sold_quantity", y="discount_per")
tvs_fig.set_axis_labels('Cantidad vendida', 'Porcentaje de descuento', fontsize=10)
tvs_fig.fig.suptitle("Televisores")

cel_fig = sns.jointplot(data=celulares, x="sold_quantity", y="discount_per")
cel_fig.set_axis_labels('Cantidad vendida', 'Porcentaje de descuento', fontsize=10)
cel_fig.fig.suptitle("Celulares")

con_fig = sns.jointplot(data=consolas, x="sold_quantity", y="discount_per")
con_fig.set_axis_labels('Cantidad vendida', 'Porcentaje de descuento', fontsize=10)
con_fig.fig.suptitle("Consolas")

air_fig = sns.jointplot(data=aires, x="sold_quantity", y="discount_per")
air_fig.set_axis_labels('Cantidad vendida', 'Porcentaje de descuento', fontsize=10)
air_fig.fig.suptitle("Aires Acondicionados")

hel_fig = sns.jointplot(data=heladeras, x="sold_quantity", y="discount_per")
hel_fig.set_axis_labels('Cantidad vendida', 'Porcentaje de descuento', fontsize=10)
hel_fig.fig.suptitle("Heladeras")

not_fig = sns.jointplot(data=notebooks, x="sold_quantity", y="discount_per")
not_fig.set_axis_labels('Cantidad vendida', 'Porcentaje de descuento', fontsize=10)
not_fig.fig.suptitle("Notebooks")

plt.show()

#### Estos gráficos permiten visualizar cómo se comportan las variables de ventas con el porcentaje de descuento, permitiendo entender cómo están correlacionadas. Lo que se puede ver es que en todas las categorías, hay muy pocos vendedores que canibalizan todas las ventas aunque no sean con el mayor descuento posible, en el caso de las notebooks por ejemplo el vendedor que vende más del doble que el segundo mayor vendedor vendió sus productos a un 15%.
#### Ignorando los vendedores outliers, se puede ver que hay vendedores que venden cantidades similares a otros, sin importar mucho cuanto es el descuento aplicado, esto lleva a pensar que el comprador no solo busca descuento, sino que pueden entrar en juego otras variables como reputación del vendedor, precio del producto, modelo, condiciones de venta y entrega, etc.

### __¿Cuáles son las marcas con mayores descuentos por categoría?__

In [ ]:
plt.figure(figsize = (15, 6))
ax = sns.boxplot(x="brand", y="discount_per", data=celulares)
ax = sns.swarmplot(x="brand", y="discount_per", data=celulares, color=".25")
ax.set_title("Marcas de celulares con descuento")
ax.set(xlabel='Marca', ylabel='Porcentaje de descuento')

In [ ]:
plt.figure(figsize = (15, 6))
ax = sns.boxplot(x="brand", y="discount_per", data=tvs)
ax = sns.swarmplot(x="brand", y="discount_per", data=tvs, color=".25")
ax.set_title("Marcas de televisores con descuento")
ax.set(xlabel='Marca', ylabel='Porcentaje de descuento')

In [ ]:
plt.figure(figsize = (18, 6))
ax = sns.boxplot(x="brand", y="discount_per", data=notebooks)
ax = sns.swarmplot(x="brand", y="discount_per", data=notebooks, color=".25")
ax.set_title("Marcas de Notebooks con descuento")
ax.set(xlabel='Marca', ylabel='Porcentaje de descuento')

#### Se puede ver que hay diferencias en las marcas que son publicadas con descuentos, habiendo algunas que tienen mayor variación  de porcentaje que otras, no siempre implica que sean las marcas más conocidas, por ejemplo Motorola ofrece la mayoría de sus productos con descuento con valores entre 15% y 20% y hay otras como BLU (menos conocida en el mercado) con mayor porcentaje de descuento.

#### Hay muchos modelos de notebooks con amplios rangos de descuentos

### __¿Existen marcas o modelos preferidos por los usuarios?__

##### Dada la naturaleza de los siguientes gráficos voy a tomar publicaciones con ciertas rangos de unidades vendidas (excluyendo las ventas outliers) para poder visualizar cómo influye el descuento en las marcas con 

#### Televisores
##### Para los televisores excluye al outlier de más de 3000 ventas, para mejorar la legibilidad

In [ ]:
ax = sns.displot(tvs[tvs.sold_quantity <= 3000], x="discount_per", y="sold_quantity", hue="brand",)
ax.set(xlabel='Porcentaje de descuento', ylabel='Ventas')
ax.fig.suptitle("Ventas de televisores según la marca y el descuento aplicado")

#### Celulares
##### Tomo publicaciones con más de 5 ventas, para poder visualizar mejor el efecto del descuento en ciertas marcas

In [ ]:
ax = sns.displot(celulares[celulares.sold_quantity > 5], x="discount_per", y="sold_quantity", hue="brand",)
ax.set(xlabel='Porcentaje de descuento', ylabel='Ventas')
ax.fig.suptitle("Ventas de celulares según la marca y el descuento aplicado")

#### Heladeras
##### Voy a tomar ventas realizadas

In [ ]:
ax = sns.displot(heladeras[heladeras.sold_quantity > 0], x="discount_per", y="sold_quantity", hue="brand",)
ax.set(xlabel='Porcentaje de descuento', ylabel='Ventas')
ax.fig.suptitle("Ventas de heladeras según la marca y el descuento aplicado")

#### A simple vista se puede ver que hay ciertas preferencias de los usuarios en las marcas, y en muchos casos el descuento no es un determinante para elegir un producto, por ejemplo, las heladeras Philco alcanzan 500 ventas con 10% de descuento, habiendo otras marcas que concretan menos ventas con mayor descuento. Esto puede ser por el factor de preferencia de marca, precio o reputación del vendedor

### __¿Cuánto gastan los usuarios? y cómo lo hacen en función de las cuotas?__

In [ ]:
ax = sns.scatterplot(data=celulares[celulares.sold_quantity > 0], x='sold_quantity', y='price', hue='installment_num')
ax.set(xlabel='Ventas', ylabel='Precio')
ax.set_title("Ventas de celulares según precio")

In [ ]:
ax = sns.scatterplot(data=notebooks[(notebooks.sold_quantity > 0) & (notebooks.sold_quantity < 3000)], x='sold_quantity', y='price', hue='installment_num')
ax.set(xlabel='Ventas', ylabel='Precio')
ax.set_title("Ventas de notebooks según precio")

In [ ]:
ax = sns.scatterplot(data=tvs[(tvs.sold_quantity > 0) & (tvs.sold_quantity < 3000)], x='sold_quantity', y='price', hue='installment_num')
ax.set(xlabel='Ventas', ylabel='Precio')
ax.set_title("Ventas de tvs según precio")

In [ ]:
ax = sns.scatterplot(data=heladeras[(heladeras.sold_quantity > 0) & (heladeras.sold_quantity < 450)], x='sold_quantity', y='price', hue='installment_num')
ax.set(xlabel='Ventas', ylabel='Precio')
ax.set_title("Ventas de heladeras según precio")

#### Esto demuestra que los usuarios muchas veces no usan los descuentos por preferir marcas más accesibles en precios, comparando con el gráfico de descuento por ventas, se ve que hay numerosas ventas de productos con más de 35% de descuento, sin embargo son productos que tienen mucho mayor valor y no son la mayoría de las ventas

#### También cabe destacar que los usuarios prefieren comprar en la mayor cantidad posible de cuotas que ofrece el vendedor, independientemente de si el precio es muy alto

### __Cuáles son las categorías favoritas para comprar con descuento?__

In [ ]:
grouped_results.sold_quantity.sum()

In [ ]:
grouped_results.sold_quantity.sum() / grouped_total.sold_quantity.sum() * 100

#### Se puede ver que no todas las categorías tienen el mismo impacto cuando se venden con descuento, puesto que por ejemplo los celulares y los aires ambos venden aproximadamente la misma cantidad de unidades, pero los usuarios prefieren comprar aires con descuento puesto que representa el 36% de las ventas

##### Aclaración: Estas conclusiones son teniendo en cuenta, que solo se levantaron 1000 productos por cada categoría

### __¿Cómo impactan las ventas con el tipo de vendedor?__

In [ ]:
ax = sns.lineplot(data=tvs[tvs.sold_quantity <= 3000], x='available_quantity', y='sold_quantity', hue='seller_status' )
ax.set(xlabel='Stock', ylabel='Ventas')
ax.set_title("Ventas de tv de vendedores según su stock (por ranking)")

In [ ]:
ax = sns.lineplot(data=celulares[celulares.sold_quantity <= 400], x='available_quantity', y='sold_quantity', hue='seller_status' )
ax.set(xlabel='Stock', ylabel='Ventas')
ax.set_title("Ventas de celulares de vendedores según su stock (por ranking)")

In [ ]:
ax = sns.lineplot(data=notebooks, x='available_quantity', y='price', hue='seller_status' )
ax.set(xlabel='Stock', ylabel='Ventas')
ax.set_title("Ventas de notebooks de vendedores según su stock (por ranking)")

#### Se puede ver que casi todos los vendedores con descuento (al menos de las primeras 100 publicaciones) son platinum, por lo que pienso que no hay gran relevancia en su seller status, por otro lado, se puede visualizar que no hay una relación directa que asocie el stock de los vendedores con sus ventas, pero si que hay una tendencia que aquellos vendedores por arriba de 200 unidades disponibles acumulan de manera lineal un crecimiento en ventas

## __¿Cómo son las ventas según el stock inicial?__

In [ ]:
ax = sns.lineplot(data=tvs[tvs.sold_quantity <= 3000], x='initial_quantity', y='sold_quantity', hue='seller_status' )
ax.set(xlabel='Stock', ylabel='Ventas')
ax.set_title("Ventas de tv de vendedores según su stock inicial (por ranking)")

In [ ]:
ax = sns.lineplot(data=celulares[celulares.sold_quantity <= 500], x='initial_quantity', y='sold_quantity', hue='seller_status' )
ax.set(xlabel='Stock', ylabel='Ventas')
ax.set_title("Ventas de Celulares de vendedores según su stock inicial (por ranking)")

In [ ]:
ax = sns.lineplot(data=heladeras, x='initial_quantity', y='sold_quantity', hue='seller_status' )
ax.set(xlabel='Stock', ylabel='Ventas')
ax.set_title("Ventas de Heladeras de vendedores según su stock inicial (por ranking)")

#### para algúnas cateogrías hay una relación mas lineal donde se destaca un incremento despues de cierta cantidad de stock inicial, parece ser una variable correlacionada con las ventas, tal vez porque indica que un vendedor es el proveedor oficial o por que salio al mercado como el primero en ofrecer el producto, al no tener las ventas históricas, estas no dejan de ser suposiciones

## __¿Cómo impacta en las ventas con descuento la cantidad de atributos que publican los vendedores y las variaciones que ofrecen?__

In [ ]:

plt.figure(figsize = (18, 5))
ax = sns.barplot(x="qty_attributes", y="sold_quantity", data=celulares, hue='qty_variations')
ax.set_title("Ventas por Cantidad de atributos de la publicacion para celulares ")
ax.set(xlabel='Cant. atributos', ylabel='Ventas')


plt.figure(figsize = (18, 5))
ax = sns.barplot(x="qty_attributes", y="sold_quantity", data=tvs, hue='qty_variations')
ax.set_title("Ventas por Cantidad de atributos de la publicacion para tvs ")
ax.set(xlabel='Cant. atributos', ylabel='Ventas')


plt.figure(figsize = (18, 5))
ax = sns.barplot(x="qty_attributes", y="sold_quantity", data=notebooks, hue='qty_variations')
ax.set_title("Ventas por Cantidad de atributos de la publicacion para notebooks ")
ax.set(xlabel='Cant. atributos', ylabel='Ventas')

#### Se puede observar que hay una relación donde mayor cantidad de información subida por los vendedores, influye en la cantidad de ventas, puede deberse por la popularidad del mismo o por que una publicación bien formateada genera confianza al comprador de lo que esta comprando y a quien.
#### También cabe remarcar que mientras mas variaciones de un producto suben, hay una crecimiento en las ventas

## __¿Cómo son las ventas por la fotos con las que se publican?__

In [ ]:
plt.figure(figsize = (18, 5))
ax = sns.lineplot(data=tvs[tvs.sold_quantity < 4000], x='pic_qty', y='sold_quantity', hue='is_official')
ax.set_title("Ventas por Cantidad de fotos de la publicacion para tvs ")
ax.set(xlabel='Cant. fotos', ylabel='Ventas')

plt.figure(figsize = (18, 5))
ax = sns.lineplot(data=heladeras[heladeras.sold_quantity < 500], x='pic_qty', y='sold_quantity', hue='is_official')
ax.set_title("Ventas por Cantidad de fotos de la publicacion para heladeras ")
ax.set(xlabel='Cant. fotos', ylabel='Ventas')

plt.figure(figsize = (18, 5))
ax = sns.lineplot(data=aires[aires.sold_quantity < 500], x='pic_qty', y='sold_quantity', hue='is_official')
ax.set_title("Ventas por Cantidad de fotos de la publicacion para celulares ")
ax.set(xlabel='Cant. fotos', ylabel='Ventas')

#### No hay una clara visualización de si la cantidad de fotos de la publicación influye en las ventas, sin embargo se ve que los vendedores no tiendas oficiales tienden a generar mas ventas que las tiendas oficiales con la misma cantidad de fotos 

In [ ]:
grouped_results.is_official.value_counts()

#### Y hay que remarcar que la cantidad de vendedores de tienda oficial y no oficial que venden con descuento en general son mas grandes los primeros que los segundos

## __Cómo es la correlación de variables para cada categoría?__

##### Solo se van a mostrar valores de correlación mayores a 0.2 y menores a -0.2

In [ ]:
corr_columns = ['price', 'installment_num', 'discount_per', 'sold_quantity', 
                                      'installment_rate', 'available_quantity', 'qty_attributes', 
                                        'initial_quantity', 'qty_variations', 'has_free_shipping', 'has_video', 'is_official']

In [ ]:
corr = tvs[(tvs.sold_quantity < 4000)][corr_columns].corr()
plt.figure(figsize=(12, 10))
ax = sns.heatmap(corr[(corr>0.2) | (corr<-0.2)], cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,annot=True, annot_kws={"size": 8}, square=True);
ax.set_title('Televisores')
plt.show()

In [ ]:
corr = notebooks[notebooks.sold_quantity < 100][corr_columns].corr()
plt.figure(figsize=(12, 10))
ax = sns.heatmap(corr[(corr>0.2) | (corr<-0.2)], cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,annot=True, annot_kws={"size": 8}, square=True);
ax.set_title('Notebooks')
plt.show()

In [ ]:
corr = heladeras[corr_columns].corr()
plt.figure(figsize=(12, 10))
ax = sns.heatmap(corr[(corr>0.2) | (corr<-0.2)], cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,annot=True, annot_kws={"size": 8}, square=True);
ax.set_title('Heladeras')
plt.show()

In [ ]:
corr = celulares[celulares.sold_quantity < 30][corr_columns].corr()
plt.figure(figsize=(12, 10))
ax = sns.heatmap(corr[(corr>0.2) | (corr<-0.2)], cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,annot=True, annot_kws={"size": 8}, square=True);
ax.set_title('Celulares')
plt.show()

#### Se puede ver que no hay una fuerte directa correlación entre las variables, los valores más altos no llegan a 0.5
##### __Nota__: se dejaron afuera los outliers de ventas que estaban muy apartados de la media